In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import ParameterGrid
from sklearn.naive_bayes import BernoulliNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('/home/jason/sentiment_data/clean.txt', sep='&&&', header=None, names=['id','sentiment','text'],
                engine='python', index_col=0)
df

,sentiment,text
id,,
399553,-,Its gonna b a lonnng day...job hunting
668139,+,Tired but motivated 'Til l8r twittheads lol
825601,+,@BethanyinFL Thank you!!! He sends his thanks...
369583,-,"David Carradine has been found dead. Awww, gr..."
761466,+,June 29th and 30th!!
...,...,...
114515,-,"@MellyG14 @kokalola : yes they were on, I hav..."
37249,-,no rafael nadal at wimbledon i dont know if i...
142521,-,If it wasn't for this exam I would be by a po...


In [3]:
df_sample = df.sample(frac=1.0)
df_sample.shape

(999993, 2)

In [4]:
def preprocess(text):
    # remove links and special characters
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())

In [5]:
df_sample['text_processed'] = df_sample['text'].apply(preprocess)

In [52]:
cv = CountVectorizer(binary=True, stop_words='english')
cv.fit(df_sample['text_processed'])
X = cv.transform(df_sample['text_processed'])

X_train, X_val, y_train, y_val = train_test_split(
    X, df_sample['sentiment'], train_size = 0.9
)

In [12]:
lr = LogisticRegression(C=100, dual=False, intercept_scaling=100, solver='lbfgs', max_iter=100, n_jobs=-1)
%timeit lr.fit(X_train, y_train)
print (accuracy_score(y_val, lr.predict(X_val)))

1min 7s ± 1.14 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
0.76965


In [66]:
cnb = ComplementNB(alpha=8.6,
                fit_prior=True,
                norm=False)
%timeit cnb.fit(X_train, y_train)
print (accuracy_score(y_val, nb.predict(X_val)))

4.42 s ± 141 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
0.76906


In [67]:
bnb = BernoulliNB(alpha=6.0, fit_prior=False)
%timeit bnb.fit(X_train, y_train)
print (accuracy_score(y_val, nb.predict(X_val)))

4.66 s ± 307 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
0.76906


In [68]:
filepath = '/home/jason/test1_public.txt'
with open(filepath, errors='surrogateescape') as f:
    test1_public = pd.DataFrame([line.split("\t", 1) for line in f], columns=['id','text'])
    test1_public['processed'] = test1_public['text'].apply(preprocess)
    
    x_test = cv.transform(test1_public['processed'])
    
    y_pred = lr.predict(x_test)
    results = pd.DataFrame(y_pred)
    results.index = test1_public['id']
    results.to_csv("jasonsnouffer_file1_entry1.txt", sep='\t', header=False)
    
    y_pred = bnb.predict(x_test)
    results = pd.DataFrame(y_pred)
    results.index = test1_public['id']
    results.to_csv("jasonsnouffer_file1_entry2.txt", sep='\t', header=False)
    
    y_pred = cnb.predict(x_test)
    results = pd.DataFrame(y_pred)
    results.index = test1_public['id']
    results.to_csv("jasonsnouffer_file1_entry3.txt", sep='\t', header=False)
    
filepath = '/home/jason/test2_public.txt'
with open(filepath, errors='surrogateescape') as f:
    test2_public = pd.DataFrame([line.split("\t", 1) for line in f], columns=['id','text'])
    test2_public['processed'] = test2_public['text'].apply(preprocess)
    
    x_test = cv.transform(test2_public['processed'])
    
    y_pred = lr.predict(x_test)
    results = pd.DataFrame(y_pred)
    results.index = test1_public['id']
    results.to_csv("jasonsnouffer_file2_entry1.txt", sep='\t', header=False)
    
    y_pred = bnb.predict(x_test)
    results = pd.DataFrame(y_pred)
    results.index = test1_public['id']
    results.to_csv("jasonsnouffer_file2_entry2.txt", sep='\t', header=False)
    
    y_pred = cnb.predict(x_test)
    results = pd.DataFrame(y_pred)
    results.index = test1_public['id']
    results.to_csv("jasonsnouffer_file2_entry3.txt", sep='\t', header=False)


ValueError: Length mismatch: Expected axis has 100000 elements, new values have 99999 elements